### 복습

In [ ]:
5월 8일 이후 오전 수업 - 텍스트마이닝(ML BASE)
Classification: Naive Bayes, Regression(Linear/Logistic), Gradient Descent
                   NN, Single Layer Perceptron, MSE, BCE, CE, Softmax, MLP(multi)->DNN
                   Embedding(NNLM, W2V), RNN, LSTM, Attention
Clustering: kMeans(Hard), GMM(Soft), PCA/LSA, LDA => Topic Modeling
Lexicon based SA(PMI, PPMI, NPMI) => 감성 정의
매일 평일 오전 9시 30분 ~ 1-2 시간 => Optional(관심있는, 초보자)
오후 => CNN + DNN(필수)

In [ ]:
IR - Indexer
Crawled data(text-unstructured) -> Preprocessing
TDM(Voca.-Dict(in-memory) | Posting-LinkedList(on-disk))
Weighting(누가 중요, 누가 덜 중요), Query Repr(TF-IDF 변형)
Ranking(Top K-retrieval) - Relevance - Similarity

Computational Model
VectorSapce Model                 Bag of Words(이론적 기반)
Concepts(Token/Term) - Dimension - Independent
vector(d) = {t1, t2, t3, ... , tn ㅌ V}, High Dim.(Sparse)
each d, 검색, kNN(분류) => Doc Embedding(token)
repr(DTM) - TDM, token1={d1,d2,d3,...,dm ㅌ C}
each t, => 비교, Term Embedding(doc)

In [ ]:
지난주까지 했던 것.

IR(텍스트 마이닝의 큰 축) - Indexer 를 했음
Crawled data(text-unstructured) -> Preprocessing
TDM(Voca.-Dict.(in-memory) | Posting-LinkedList(on-disk))
Weighting(누가 중요, 누가 덜 중요), Query Repr(TF-IDF 변형)
Ranking(Top K-retrieval) - Relevance - Similarity
여기까지 하면 우리가 가진 문서를 가지고 주어진 Query를 통해 Projection한 document끼리 얼마만큼의 유사성을 가지는지 알 수 있음
여기까지의 근간은,

Computataional Model
VectorSpace Model                  Bag of Words(이론적 기반)
Concepts(Token/Term) - Dimension - Independent
vector(doc) = {t1, t2, t3, ..., tn ㅌ V}, High Dim. Sparse 형태의 벡터 표현이 만들어짐. 이를 통해,
each doc 비교가 가능해짐. 각각의 doc 들을 벡터를 가지고 비교 가능. 그 중 하나가 검색. 오늘 kNN 으로 분류까지 해볼 것. => Doc Embedding
-> token/term으로 임베딩

rper(DTM) - TDM 은 서로 .T할수 있는 관계. 이를 이용하면, token1 = {d1, d2, d3, ..., dm ㅌ C}
each t, 비교 가능. => Term Embedding(doc으로 임베딩); 동일한 문서에, 동일한 문맥에 같이 나오는 애들.
단어들끼리 문맥을 기반으로 해서 의미적 유사성을 찾아낼 수 있음.

### 수업

In [1]:
from collections import Counter
from struct import pack, unpack

In [2]:
!ls *.dat

C.dat       TDM.dat     V.dat       posting.dat weights.dat


In [19]:
import json
# 지난주 금요일 indexer 이어서
with open('C.dat', 'r', encoding='utf8') as fp:
    C = json.load(fp)
# list => [docindex:{path, maxfre, length}, ...]    
with open('V.dat', 'r', encoding='utf8') as fp:
    V = json.load(fp)
# contrlled vocabulary(normalization 으로 몇 개 제거해서). list => (단어, 단어ㅏ, ...)
with open('TDM.dat', 'r') as fp:
    TDM = json.load(fp)
# InvertedIndex TDM => dict, (V.idx: {docfreq, pos})

In [21]:
C

[{'path': 'news_naver/0004983292sid104.txt', 'maxfreq': 41, 'length': 0},
 {'path': 'news_naver/0004160116cid1089303.txt', 'maxfreq': 81, 'length': 0},
 {'path': 'news_naver/0013818156sid100.txt', 'maxfreq': 41, 'length': 0},
 {'path': 'news_naver/0003752234sid101.txt', 'maxfreq': 42, 'length': 0},
 {'path': 'news_naver/0001861925sid103.txt', 'maxfreq': 35, 'length': 0},
 {'path': 'news_naver/0003485660sid102.txt', 'maxfreq': 47, 'length': 0},
 {'path': 'news_naver/0003752127sid102.txt', 'maxfreq': 39, 'length': 0},
 {'path': 'news_naver/0001260297cid1085809.txt', 'maxfreq': 110, 'length': 0},
 {'path': 'news_naver/0011447593sid105.txt', 'maxfreq': 41, 'length': 0},
 {'path': 'news_naver/0006689761sid104.txt', 'maxfreq': 44, 'length': 0},
 {'path': 'news_naver/0003346983sid101.txt', 'maxfreq': 41, 'length': 0},
 {'path': 'news_naver/0004863046sid101.txt', 'maxfreq': 49, 'length': 0},
 {'path': 'news_naver/0006690086sid100.txt', 'maxfreq': 60, 'length': 0},
 {'path': 'news_naver/0013821

In [22]:
V

['1박2일',
 '일정으로',
 '일본을',
 '방문한',
 '윤석열',
 '대통령이',
 '17일',
 '도쿄',
 '호텔에서',
 '열린',
 '일한',
 '친선단체',
 '일본',
 '정계',
 '지도자를',
 '접견에서',
 '일한의원연맹',
 '회장에',
 '취임하는',
 '스가',
 '요시히데',
 '총리와',
 '인사하고',
 '있다',
 '연합뉴스',
 '김경민',
 '특파원',
 '방일',
 '중인',
 '정치권',
 '인사들과',
 '만나',
 '한일',
 '갈등',
 '주요',
 '현안에',
 '대해',
 '논의했다',
 '제1야당인',
 '입헌민주당의',
 '이즈미',
 '겐타',
 '대표는',
 '이날',
 '대통령과',
 '레이더',
 '초계기',
 '문제와',
 '위안부',
 '소녀상',
 '건립',
 '문제',
 '등을',
 '언급했다고',
 '밝혔다',
 '그는',
 '대통령의',
 '한일관계',
 '개선',
 '노력에',
 '경의를',
 '표하고',
 '아직',
 '양국',
 '사이에',
 '다양한',
 '현안이',
 '있기',
 '때문에',
 '그런',
 '것도',
 '해결될',
 '있게',
 '노력해주시고',
 '저희도',
 '협력하겠다고',
 '말씀드렸다',
 '전했다',
 '조사와',
 '위안부상',
 '언급에',
 '반응을',
 '묻자',
 '구체적인',
 '답변은',
 '없었다',
 '말했다',
 '다만',
 '대통령은',
 '납북',
 '문제에',
 '대해서는',
 '한국에도',
 '피해자가',
 '있고',
 '이산가족',
 '등도',
 '함께',
 '노력하자는',
 '취지로',
 '말했다고',
 '일본의',
 '야당과도',
 '이렇게',
 '의견을',
 '교환하는',
 '자세에',
 '다시금',
 '진심을',
 '느꼈다',
 '평가했다',
 '아울러',
 '초당파',
 '의원',
 '모임인',
 '일한의원연맹도',
 '대통령에게',
 '후쿠시마',
 '제1원자력발전소',
 '오염수',
 '해양',
 '방류와',


In [23]:
TDM

{'0': {'docfreq': 9, 'pos': 9269664},
 '1': {'docfreq': 15, 'pos': 11656380},
 '2': {'docfreq': 25, 'pos': 11654880},
 '3': {'docfreq': 16, 'pos': 11654892},
 '4': {'docfreq': 83, 'pos': 11654868},
 '5': {'docfreq': 54, 'pos': 11655060},
 '6': {'docfreq': 145, 'pos': 11757420},
 '7': {'docfreq': 45, 'pos': 11656416},
 '8': {'docfreq': 5, 'pos': 8378712},
 '9': {'docfreq': 83, 'pos': 11598576},
 '10': {'docfreq': 37, 'pos': 11788296},
 '11': {'docfreq': 4, 'pos': 8007828},
 '12': {'docfreq': 130, 'pos': 11710284},
 '13': {'docfreq': 6, 'pos': 11735736},
 '14': {'docfreq': 3, 'pos': 5438988},
 '15': {'docfreq': 2, 'pos': 5439000},
 '16': {'docfreq': 4, 'pos': 5439012},
 '17': {'docfreq': 4, 'pos': 5439024},
 '18': {'docfreq': 3, 'pos': 5439036},
 '19': {'docfreq': 63, 'pos': 11861712},
 '20': {'docfreq': 6, 'pos': 9630684},
 '21': {'docfreq': 21, 'pos': 10941276},
 '22': {'docfreq': 5, 'pos': 5795232},
 '23': {'docfreq': 307, 'pos': 11861136},
 '24': {'docfreq': 108, 'pos': 11787240},
 '

In [24]:
# TF-IDF => 가중치 계산
from math import log

tf1 = lambda freq: 1 if freq >0 else 0    # binary
tf2 = lambda freq: freq                   # raw count
tf3 = lambda freq, sumfreq: freq/sumfreq  # term freq
tf4 = lambda freq: log(1+freq, 10)            # log normalization
tf5 = lambda freq, maxfreq: tf6(.5,freq,maxfreq)           # double normalization .5
tf6 = lambda k, freq, maxfreq: k + (1-k) * (freq/maxfreq)  # double normalization K

idf1 = lambda:1
idf2 = lambda df, N:log(N/df) 
idf3 = lambda df, N:log(N/(1+df))+1
idf4 = lambda df, maxdf:log(maxdf/(1+df))
idf5 = lambda df, N:log((N-df)/df)

In [25]:
# Posting 구조 잘 불러오는지 확인
with open('posting.dat', 'rb') as fp2:
    fp2_pos = TDM['20']['pos']
    while fp2_pos != -1:
        fp2.seek(fp2_pos)
        info = unpack('iii', fp2.read(12))
        fp2_pos = info[-1]
        print(V[20], info)
        
# 나오는 결과는 단어 (단어가 나오는 문서번호, 그 문서에서 단어의 빈도, 위치)

요시히데 (334, 1, 5439060)
요시히데 (186, 2, 4640976)
요시히데 (159, 1, 1628628)
요시히데 (58, 1, 823032)
요시히데 (27, 1, 240)
요시히데 (0, 2, -1)


In [26]:
# Documents 가중치 계산하는, Documents vector length 누적

open('weights.dat', 'wb').close()

# TF(특정 d에서 t의 freq=>Posting, 특정 d에서의 maxfreq=>C[문서idx]['maxfreq'])
# IDF(t의 docfreq=>TDM[termidx][docfreq], 전체 d의 갯수=>len(C))

for j, termInfo in TDM.items(): # termInfo={'docfreq':1, 'pos':pos}
    fp1 = open('weights.dat', 'ab')  # 구한 weight 값 저장하는 부분
    fp1_pos = fp1.tell()
    # pos:(문서idx, weight), (문서idx, weight), (문서idx, weight), ...
    
    with open('posting.dat', 'rb') as fp2:
        # j번째 단어의 정보 TDM 에서 dict 가져오고
        # j번째 단어의 빈도가 기록된 posting의 시작 위치
        fp2_pos = termInfo['pos']
        # 마지막까지 : -1
        while fp2_pos != -1:
            # 파일을 열면 커서가 파일의 마지막에 r,w로 열면 0번째 위치, a면 커서 마지막
            # r이니까 시작위치를 이동
            fp2.seek(fp2_pos)
            # posting (문서idx, 빈도, 다음위치)
            info = unpack('iii', fp2.read(12)) # (docId, freq, next_pos)
            fp2_pos = info[-1]  # 다음위치 바꿔준 것
            
            TF = tf6(0, info[1], C[info[0]]['maxfreq'])  # k =0 이면 normalization 한번
            IDF = idf2(termInfo['docfreq'], len(C))
            W = TF*IDF # => a*TF*(1-a)*IDF
            
            fp1.write(pack('if', info[0], W))  # (문서idx, weight)를 while 문 만큼 저장. 처음 시작한 위치만 기록
            C[info[0]]['length'] += W**2
            # 특정 doc 에서 나온 모든 term 들의 가중치를 누적만 해놓은 것. distance 구하는 것인데, sqrt 빼고.
            
    TDM[j]['pos'] = fp1_pos  # TDM 의 Posting 이 weight 의 pos
    fp1.close()
    
# 이게 지난번에 했던 WDM 이랑 변수만 다르고 똑같은 것.

In [27]:
C

[{'path': 'news_naver/0004983292sid104.txt',
  'maxfreq': 41,
  'length': 40.362755120585234},
 {'path': 'news_naver/0004160116cid1089303.txt',
  'maxfreq': 81,
  'length': 72.68959601807062},
 {'path': 'news_naver/0013818156sid100.txt',
  'maxfreq': 41,
  'length': 31.95224943958833},
 {'path': 'news_naver/0003752234sid101.txt',
  'maxfreq': 42,
  'length': 31.839377466412046},
 {'path': 'news_naver/0001861925sid103.txt',
  'maxfreq': 35,
  'length': 37.02158161804877},
 {'path': 'news_naver/0003485660sid102.txt',
  'maxfreq': 47,
  'length': 36.27529536625824},
 {'path': 'news_naver/0003752127sid102.txt',
  'maxfreq': 39,
  'length': 79.50612455217758},
 {'path': 'news_naver/0001260297cid1085809.txt',
  'maxfreq': 110,
  'length': 59.68223067808069},
 {'path': 'news_naver/0011447593sid105.txt',
  'maxfreq': 41,
  'length': 17.868344224551514},
 {'path': 'news_naver/0006689761sid104.txt',
  'maxfreq': 44,
  'length': 22.692897194669335},
 {'path': 'news_naver/0003346983sid101.txt',
  

In [29]:
json.dumps({1:'a'})

'{"1": "a"}'

In [30]:
json.loads(json.dumps({1:'a'}))

{'1': 'a'}

In [35]:
# Weight 잘 계산해서 저장됐는지 확인. 역순으로 idx 숫자가 줄어들면 제대로

idx = '1' # Dict로 저장해서 int -> str 로 바뀜(json 문법)
pos = TDM[idx]['pos']
df = TDM[idx]['docfreq']

with open('weights.dat', 'rb') as fp:
    for i in range(df):
        fp.seek(pos+(8*i))
        info = unpack('if', fp.read(8))
        print(V[int(idx)], info)

일정으로 (409, 0.0707964077591896)
일정으로 (322, 0.08531875163316727)
일정으로 (321, 0.053668245673179626)
일정으로 (301, 0.06790675967931747)
일정으로 (272, 0.07394291460514069)
일정으로 (200, 0.07562343776226044)
일정으로 (186, 0.13048750162124634)
일정으로 (144, 0.07922455668449402)
일정으로 (120, 0.04321339353919029)
일정으로 (106, 0.014467092230916023)
일정으로 (96, 0.14467091858386993)
일정으로 (87, 0.0168905146420002)
일정으로 (64, 0.08115685731172562)
일정으로 (24, 0.06790675967931747)
일정으로 (0, 0.08115685731172562)


In [32]:
# 설명 그림 삽입

In [ ]:
# Query 해보자

In [36]:
len(C), len(V)

(418, 186232)

In [37]:
import re
from nltk.tokenize import sent_tokenize, word_tokenize
from konlpy.tag import Komoran 

p1 = re.compile('[a-zA-Z0-9_-]+@(?:[.]?[a-zA-Z0-9_-]+)+')
p2 = re.compile('[{}]'.format(re.escape('''"'=ⓒ?!【】''')))
p3 = re.compile('[\(\[].+?[\)\]]')
p4 = re.compile('[a-zA-Z]+')
p5 = re.compile('\s+')
p6 = re.compile('[^ㄱ-하-ㅣ가-힣0-9 .]') # 한글, 숫자, 띄어쓰기만 남을 것. '.'을 빼버리면 문장이 어디서 끝나는지 모름
p7 = re.compile('\s[.]')

# 품사로 불용어 처리
stopwords = ['EC', 'EF', 'EP', 'ETM', 'ETN', 'JC', 'JKB', 'JKC', 'JKG', 'JKO', 'JKQ', 'JKS', 'JKV',
             'JX', 'XPN', 'XR', 'XSA', 'XSN', 'XSV']

# 형태소 분석기
ma = Komoran()

# 정규식
def preprocessing1(d, *patterns):
    for i in range(len(patterns)):
        d = patterns[i].sub(' ', d)
    return d.strip()

def preprocessing2(d):
    sentences = sent_tokenize(d)
    words = word_tokenize(d)
    morphems = list()
    nouns = list()
    pos = list()  # 품사
    for s in sentences:
        for t in ma.pos(s):  # (형태소, 품사)
            morphems.append(t[0])
            if t[1][0] == 'N':
                nouns.append(t[0])
            pos.append('/'.join(t))

    bigram1 = ngram1(' '.join(morphems))
    bigram2 = ngram2(d)
    
    return sentences+words+morphems+nouns+pos+bigram1+bigram2

# Normalization
def preprocessing3(tokens):
    under = 1
    upper = 10  # 글자수 threshold
    result1 = list()
    # 길이로 제한
    for t in tokens:
        if under < len(t) < upper: # sentences 날라가고, bigram1 중 너무 긴애들 날라가고, 1음절 형태소 날라갈 것. 띄어쓰기 잘못도
            result1.append(t)
    # 품사로 제한
    result2 = list()
    for t in result1:
        if len(t.split('/')) == 2:
            if t.split('/')[-1] not in stopwords:
                result2.append(t)
        else:
            result2.append(t)
    return result2


def ngram1(t, n=2):  # Improved Bow (토큰1 토큰2 => 토큰)
    tokens = t.split()
    gram = list()
    
    if len(tokens) < n:
        return tokens
        
    for i in range(len(tokens)-(n-1)):
        gram.append(' '.join(tokens[i:i+n]))
    return gram

def ngram2(t, n=2):  # Subword (가나다라마바사 => 나다, 다라)
    tokens = list(t)
    gram = list()
    
    if len(tokens) < n:
        return tokens
        
    for i in range(len(tokens)-(n-1)):
        gram.append(''.join(tokens[i:i+n]))
    return gram

In [38]:
!ls *.dat

C.dat       TDM.dat     V.dat       posting.dat weights.dat


In [39]:
#Q = ''''''
with open(C[0]['path']) as fp:
    Q = fp.read()

d = preprocessing1(Q, p1, p2, p3, p4, p6, p5, p7, p5)
tokens = preprocessing2(d)
QTM = Counter(preprocessing3(tokens))
maxfreq = max(QTM.values())
QLength = 0

for t, freq in QTM.items():
    TF = tf6(0, freq, maxfreq)
    IDF = idf2(TDM[str(V.index(t))]['docfreq'], len(C))
    W = TF*IDF
    QLength += W**2

In [40]:
TDM

{'0': {'docfreq': 9, 'pos': 0},
 '1': {'docfreq': 15, 'pos': 80},
 '2': {'docfreq': 25, 'pos': 200},
 '3': {'docfreq': 16, 'pos': 400},
 '4': {'docfreq': 83, 'pos': 528},
 '5': {'docfreq': 54, 'pos': 1192},
 '6': {'docfreq': 145, 'pos': 1624},
 '7': {'docfreq': 45, 'pos': 2784},
 '8': {'docfreq': 5, 'pos': 3144},
 '9': {'docfreq': 83, 'pos': 3184},
 '10': {'docfreq': 37, 'pos': 3848},
 '11': {'docfreq': 4, 'pos': 4144},
 '12': {'docfreq': 130, 'pos': 4176},
 '13': {'docfreq': 6, 'pos': 5216},
 '14': {'docfreq': 3, 'pos': 5264},
 '15': {'docfreq': 2, 'pos': 5288},
 '16': {'docfreq': 4, 'pos': 5304},
 '17': {'docfreq': 4, 'pos': 5336},
 '18': {'docfreq': 3, 'pos': 5368},
 '19': {'docfreq': 63, 'pos': 5392},
 '20': {'docfreq': 6, 'pos': 5896},
 '21': {'docfreq': 21, 'pos': 5944},
 '22': {'docfreq': 5, 'pos': 6112},
 '23': {'docfreq': 307, 'pos': 6152},
 '24': {'docfreq': 108, 'pos': 8608},
 '25': {'docfreq': 1, 'pos': 9472},
 '26': {'docfreq': 19, 'pos': 9480},
 '27': {'docfreq': 33, 'pos

In [41]:
QTM

Counter({'1박2일': 1,
         '일정으로': 1,
         '일본을': 1,
         '방문한': 1,
         '윤석열': 6,
         '대통령이': 2,
         '17일': 2,
         '도쿄': 8,
         '호텔에서': 1,
         '열린': 2,
         '일한': 9,
         '친선단체': 1,
         '일본': 25,
         '정계': 8,
         '지도자를': 1,
         '접견에서': 1,
         '일한의원연맹': 3,
         '회장에': 2,
         '취임하는': 1,
         '스가': 8,
         '요시히데': 2,
         '총리와': 1,
         '인사하고': 1,
         '있다': 2,
         '연합뉴스': 3,
         '김경민': 9,
         '특파원': 3,
         '방일': 4,
         '중인': 2,
         '정치권': 1,
         '인사들과': 1,
         '만나': 4,
         '한일': 15,
         '갈등': 4,
         '주요': 7,
         '현안에': 1,
         '대해': 5,
         '논의했다': 1,
         '제1야당인': 1,
         '입헌민주당의': 1,
         '이즈미': 9,
         '겐타': 4,
         '대표는': 3,
         '이날': 12,
         '대통령과': 1,
         '레이더': 6,
         '초계기': 3,
         '문제와': 1,
         '위안부': 3,
         '소녀상': 3,
         '건립': 4,
         '문제': 14,
    

In [42]:
#Q = ''''''
with open(C[0]['path']) as fp:
    Q = fp.read()

d = preprocessing1(Q, p1, p2, p3, p4, p6, p5, p7, p5)
tokens = preprocessing2(d)
QTM = Counter(preprocessing3(tokens))
maxfreq = max(QTM.values())
QLength = 0

for t, freq in QTM.items():
    TF = tf6(0, freq, maxfreq)
    IDF = idf2(TDM[str(V.index(t))]['docfreq'], len(C))
    W = TF*IDF
    QLength += W**2
    QTM[t] = W

In [43]:
QTM

Counter({'1박2일': 0.093616020858257,
         '일정으로': 0.08115685930298894,
         '일본을': 0.06869769774772086,
         '방문한': 0.07958274903134085,
         '윤석열': 0.23658158410655974,
         '대통령이': 0.09982914077856009,
         '17일': 0.05164622878556985,
         '도쿄': 0.434891501025256,
         '호텔에서': 0.10795228097782088,
         '열린': 0.07886052803551992,
         '일한': 0.5322212604615802,
         '친선단체': 0.11339480661963088,
         '일본': 0.7121627939446182,
         '정계': 0.8280433099115516,
         '지도자를': 0.12041144253308894,
         '접견에서': 0.13030083541377588,
         '일한의원연맹': 0.3401844198588926,
         '회장에': 0.22678961323926175,
         '취임하는': 0.12041144253308894,
         '스가': 0.3692383816845315,
         '요시히데': 0.2070108274778879,
         '총리와': 0.07295021938539838,
         '인사하고': 0.10795228097782088,
         '있다': 0.015055301704271176,
         '연합뉴스': 0.09902562478540511,
         '김경민': 1.324861777871288,
         '특파원': 0.22617383805060848,
     

In [44]:
from math import sqrt
# inner product
K = 10
candidates = dict()

for t, qw in QTM.items():
    j = str(V.index(t))
    info = TDM[j]
    
    with open('weights.dat', 'rb') as fp:
        for i in range(info['docfreq']):
            fp.seek(info['pos']+(8*i))
            docinfo = unpack('if', fp.read(8))
            innerproduct = qw * docinfo[1]
            
            if docinfo[0] not in candidates:
                candidates[docinfo[0]] = innerproduct
            else:
                candidates[docinfo[0]] += innerproduct
            
for i, ip in candidates.items():
    candidates[i] = ip/(sqrt(QLength)*sqrt(C[i]['length']))
    
sorted(candidates.items(), key=lambda c:c[1], reverse=True)[:K]
# 0번 문서 뒤진거라 아래 결과 나옴

[(0, 0.999782867376097),
 (186, 0.38638804377874264),
 (145, 0.3638093165719859),
 (159, 0.2782361367492827),
 (48, 0.2746618193736111),
 (27, 0.25969318629381205),
 (58, 0.25344799868757395),
 (221, 0.19915320871875733),
 (100, 0.1883591605538512),
 (370, 0.16724537620585309)]

In [45]:
# inner product
K = 10
candidates = dict()

for t, qw in QTM.items():
    j = str(V.index(t))
    info = TDM[j]
    
    with open('weights.dat', 'rb') as fp:
        for i in range(info['docfreq']):
            fp.seek(info['pos']+(8*i))
            docinfo = unpack('if', fp.read(8))
            innerproduct = qw * docinfo[1]
            
            if docinfo[0] not in candidates:
                candidates[docinfo[0]] = innerproduct
            else:
                candidates[docinfo[0]] += innerproduct
            
for i, ip in candidates.items():
    candidates[i] = ip/(sqrt(QLength)*sqrt(C[i]['length']))
    
res = sorted(candidates.items(), key=lambda c:c[1], reverse=True)[:K]
for r in res:
    print(C[r[0]]['path'], r[1])  # r[1] 이 가중치
    with open(C[r[0]]['path'], 'r', encoding='utf8') as fp:
        print('\n'.join(sent_tokenize(fp.read())[0:2]), end='\n\n')

news_naver/0004983292sid104.txt 0.999782867376097
1박2일 일정으로 일본을 방문한 윤석열 대통령이 17일 도쿄 한 호텔에서 열린 일한 친선단체 및 일본 정계 지도자를 접견에서 일한의원연맹 회장에 취임하는 스가 요시히데 전 총리와 인사하고 있다 .
연합뉴스 【도쿄=김경민 특파원】 방일 중인 윤석열 대통령이 17일 일본 정치권 인사들과 만나 한일 갈등 주요 현안에 대해 논의했다.

news_naver/0003346923sid100.txt 0.38638804377874264
방일 마지막날 ‘한일 비즈니스라운드테이블’ 참석



한일 비즈니스라운드 테이블 입장하는 윤석열 대통령 - 1박2일간의 일정으로 일본을 방문한 윤석열 대통령이 17일 도쿄 게이단렌(일본경제단체연합회) 회관에서 열린 한일 비즈니스라운드 테이블에서 도쿠라 마사카즈 게이단렌 회장과 함께 입장하고 있다.-연합뉴스윤석열 대통령은 일본 방문 이틀째이자 방일 마지막날인 17일 일본 정치인과 재계 인사들을 연이어 만나 한일관계 복원 의지를 재확인했다.윤 대통령은 이날 도쿄 게이단렌(일본 경제단체연합회) 회관에서 열린 ‘한일 비즈니스 라운드테이블’에 참석해 한일 경제인들과 교류했다.
윤 대통령은 행사 인사말에서 “두 나라는 공급망, 기후변화, 첨단과학기술, 경제안보 등 다양한 글로벌 어젠다에 대해 공동으로 협력하고 대응할 것”이라며 “디지털 전환, 반도체, 배터리, 전기차 등 미래첨단 신산업 분야에서 양국 간 협력의 여지가 매우 크다”고 말했다.

news_naver/0004983295sid100.txt 0.3638093165719859
외교부에 협조 요청  [파이낸셜뉴스] 윤석열 대통령이 한일 군사정보보호협정(GSOMIA, 지소미아) 정상화를 선언한 지 하루 만인 17일 국방부가 이에 필요한 절차에 착수한 것으로 전해졌다.
[그래픽] 한일 군사정보보호협정(GSOMIA) 대북정보 제공 개요 (서울=연합뉴스) 반종빈 기자 = 윤석열 대통령과 기시다 후미오 일본 총리의 16일

In [46]:
# inner product
K = 10
candidates = dict()

for t, qw in QTM.items():
    j = str(V.index(t))
    info = TDM[j]
    
    with open('weights.dat', 'rb') as fp:
        for i in range(info['docfreq']):
            fp.seek(info['pos']+(8*i))
            docinfo = unpack('if', fp.read(8))
            innerproduct = qw * docinfo[1]
            
            if docinfo[0] not in candidates:
                candidates[docinfo[0]] = innerproduct
            else:
                candidates[docinfo[0]] += innerproduct
            
for i, ip in candidates.items():
    candidates[i] = ip/(sqrt(QLength)*sqrt(C[i]['length']))
    
res = sorted(candidates.items(), key=lambda c:c[1], reverse=True)[:K]
for i, r in enumerate(res):
    print((i+1), C[r[0]]['path'], r[1])  # r[1] 이 가중치
    with open(C[r[0]]['path'], 'r', encoding='utf8') as fp:
        print('\n'.join(sent_tokenize(fp.read())[0:2]), end='\n\n')

1 news_naver/0004983292sid104.txt 0.999782867376097
1박2일 일정으로 일본을 방문한 윤석열 대통령이 17일 도쿄 한 호텔에서 열린 일한 친선단체 및 일본 정계 지도자를 접견에서 일한의원연맹 회장에 취임하는 스가 요시히데 전 총리와 인사하고 있다 .
연합뉴스 【도쿄=김경민 특파원】 방일 중인 윤석열 대통령이 17일 일본 정치권 인사들과 만나 한일 갈등 주요 현안에 대해 논의했다.

2 news_naver/0003346923sid100.txt 0.38638804377874264
방일 마지막날 ‘한일 비즈니스라운드테이블’ 참석



한일 비즈니스라운드 테이블 입장하는 윤석열 대통령 - 1박2일간의 일정으로 일본을 방문한 윤석열 대통령이 17일 도쿄 게이단렌(일본경제단체연합회) 회관에서 열린 한일 비즈니스라운드 테이블에서 도쿠라 마사카즈 게이단렌 회장과 함께 입장하고 있다.-연합뉴스윤석열 대통령은 일본 방문 이틀째이자 방일 마지막날인 17일 일본 정치인과 재계 인사들을 연이어 만나 한일관계 복원 의지를 재확인했다.윤 대통령은 이날 도쿄 게이단렌(일본 경제단체연합회) 회관에서 열린 ‘한일 비즈니스 라운드테이블’에 참석해 한일 경제인들과 교류했다.
윤 대통령은 행사 인사말에서 “두 나라는 공급망, 기후변화, 첨단과학기술, 경제안보 등 다양한 글로벌 어젠다에 대해 공동으로 협력하고 대응할 것”이라며 “디지털 전환, 반도체, 배터리, 전기차 등 미래첨단 신산업 분야에서 양국 간 협력의 여지가 매우 크다”고 말했다.

3 news_naver/0004983295sid100.txt 0.3638093165719859
외교부에 협조 요청  [파이낸셜뉴스] 윤석열 대통령이 한일 군사정보보호협정(GSOMIA, 지소미아) 정상화를 선언한 지 하루 만인 17일 국방부가 이에 필요한 절차에 착수한 것으로 전해졌다.
[그래픽] 한일 군사정보보호협정(GSOMIA) 대북정보 제공 개요 (서울=연합뉴스) 반종빈 기자 = 윤석열 대통령과 기시다 후미오 일본 총

In [47]:
# inner product
K = 10
candidates = dict()

for t, qw in QTM.items():
    j = str(V.index(t))
    info = TDM[j]
    
    with open('weights.dat', 'rb') as fp:
        for i in range(info['docfreq']):
            fp.seek(info['pos']+(8*i))
            docinfo = unpack('if', fp.read(8))
            innerproduct = qw * docinfo[1]
            
            if docinfo[0] not in candidates:
                candidates[docinfo[0]] = innerproduct
            else:
                candidates[docinfo[0]] += innerproduct
            
for i, ip in candidates.items():
    candidates[i] = ip/(sqrt(QLength)*sqrt(C[i]['length']))
    
res = sorted(candidates.items(), key=lambda c:c[1], reverse=True)[:K]
for i, r in enumerate(res):
    print('{}. {} ({})'.format((i+1), C[r[0]]['path'], r[1]))  # r[1] 이 가중치
    with open(C[r[0]]['path'], 'r', encoding='utf8') as fp:
        print('\n'.join(
            sent_tokenize(
                preprocessing1(fp.read(), p6, p5, p7, p5))[0:2]),
              end='\n\n')

1. news_naver/0004983292sid104.txt (0.999782867376097)
1박2일 일정으로 일본을 방문한 윤석열 대통령이 17일 도쿄 한 호텔에서 열린 일한 친선단체 및 일본 정계 지도자를 접견에서 일한의원연맹 회장에 취임하는 스가 요시히데 전 총리와 인사하고 있다 연합뉴스 도쿄 김경민 특파원 방일 중인 윤석열 대통령이 17일 일본 정치권 인사들과 만나 한일 갈등 주요 현안에 대해 논의했다.
일본 제1야당인 입헌민주당의 이즈미 겐타 대표는 이날 윤 대통령과 레이더 초계기 문제와 위안부 소녀상 건립 문제 등을 언급했다고 밝혔다.

2. news_naver/0003346923sid100.txt (0.38638804377874264)
방일 마지막날 한일 비즈니스라운드테이블 참석 한일 비즈니스라운드 테이블 입장하는 윤석열 대통령 - 1박2일간의 일정으로 일본을 방문한 윤석열 대통령이 17일 도쿄 게이단렌 일본경제단체연합회 회관에서 열린 한일 비즈니스라운드 테이블에서 도쿠라 마사카즈 게이단렌 회장과 함께 입장하고 있다.-연합뉴스윤석열 대통령은 일본 방문 이틀째이자 방일 마지막날인 17일 일본 정치인과 재계 인사들을 연이어 만나 한일관계 복원 의지를 재확인했다.윤 대통령은 이날 도쿄 게이단렌 일본 경제단체연합회 회관에서 열린 한일 비즈니스 라운드테이블 에 참석해 한일 경제인들과 교류했다.
윤 대통령은 행사 인사말에서 두 나라는 공급망 기후변화 첨단과학기술 경제안보 등 다양한 글로벌 어젠다에 대해 공동으로 협력하고 대응할 것 이라며 디지털 전환 반도체 배터리 전기차 등 미래첨단 신산업 분야에서 양국 간 협력의 여지가 매우 크다 고 말했다.

3. news_naver/0004983295sid100.txt (0.3638093165719859)
외교부에 협조 요청 파이낸셜뉴스 윤석열 대통령이 한일 군사정보보호협정 지소미아 정상화를 선언한 지 하루 만인 17일 국방부가 이에 필요한 절차에 착수한 것으로 전해졌다.
그래픽 한일 군사정보보호협정 대북정보 제공

In [ ]:
# 변형

In [48]:
# with open(C[0]['path']) as fp:
#     Q = fp.read()
# Ad-hoc (data-driven)

# Unkown Data? 동작(ML:Genreralizaion)
Q = '''북한이 지난달 21∼23일 ‘해일’과 25∼27일 ‘해일-1형’에 이어 이달 4∼7일에도 ‘해일-2형’ 핵무인수중공격정(핵어뢰)의 수중 폭발시험을 진행했다고 8일 공개했다. 보름 새 세 차례나 핵어뢰의 수중 폭발시험 성공을 과시한 것. 11년간 개발한 ‘비밀병기’의 다종다양화와 실전배치가 임박했다는 경고를 통해 우리 군의 과장·조작 가능성을 맞받아친 것으로 풀이된다.

북한 조선중앙통신은 8일 “해일-2형이 1000km 거리를 모의해 조선 동해에 설정된 타원 및 8자형 침로를 71시간 6분간 잠항해 7일 오후 목표가상수역인 함남 단천시 룡대항 앞바다에 도달했으며 시험용전투부가 정확히 수중 기폭됐다”고 보도했다.

해일-2형의 잠항 거리(1000km)는 해일-1형(600km)보다 400km나 늘어났다. 북한 최북단 해역에서 쏴도 한국의 남·동해안의 모든 항구가 타격권에 들어간다. 한미 해군의 감시망을 피해 공해상을 우회해 남부 해안까지 접근할 수 있을 것으로도 추정된다. 잠항 시간(71시간 6분)도 최대 30시간가량 길어졌다. 대용량 배터리를 장착해 더 오래 물속에 머물면서 먼 거리의 표적을 핵타격할 수 있도록 개량한 것으로 보인다. 북한 매체가 공개한 해일-2형은 해일-1형보다 동체가 다소 길어진 것으로 추정된다. 외관 색상도 검은색으로 해일-1형(빨간색)과 달랐다.

김동엽 북한대학원대 교수는 “잠항 거리 1000km라면 북한 항구를 출발해 일본 항구까지 충분히 도달할 수 있고, 수상 함정을 이용하면 괌도 불가능하지 않을 것”이라며 “항구뿐 아니라 원거리 항모단이나 상륙강습단을 은밀히 공격할 수도 있다”고 말했다.

해일-2형의 시험 타이밍(4∼7일)도 한미일 3국 견제 목적임이 뚜렷이 드러났다. 4일 미국의 핵추진 항공모함 니미츠 등 한미일 3국 해군 전력이 제주 남쪽 공해상에서 북한 핵어뢰 침투를 상정한 대잠전 훈련을 벌였다. 5일에는 B-52H 전략폭격기가 날아와 우리 군의 F-35A 스텔스전투기 등과 함께 한미 연합 공중훈련을 실시했다.
'''

# DL, ML = dataloader(yield, preprocessing)
d = preprocessing1(Q, p1, p2, p3, p4, p6, p5, p7, p5)
tokens = preprocessing2(d)
# 내가 학습을 위처럼 시켰으니 앞으로 올 새로운 문서(쿼리)도 똑같은 처리를 해줘야 같은 벡터 공간에 생길 것.

QTM = Counter(filter(lambda t:t in V, preprocessing3(tokens)))
maxfreq = max(QTM.values())
QLength = 0

for t, freq in QTM.items():
    TF = tf6(0, freq, maxfreq)  # k를 0.5로 바꿀 수 있음. 이전 결과가 1이 나옴을 확인했기 때문. 0.5를 주면 빈도의 영향을 덜 받게 됨
    # 그래서 IDF 의 영향력을 키워주는 것.
    IDF = idf2(TDM[str(V.index(t))]['docfreq'], len(C))
    W = TF*IDF
    QLength += W**2
    QTM[t] = W

In [49]:
# inner product
K = 10
candidates = dict()

for t, qw in QTM.items():  # 쿼리에 대한 가중치 들어있음
    j = str(V.index(t))
    info = TDM[j]
    
    # 각 단어에 있는 문서의 웨이트들 돌면서,
    with open('weights.dat', 'rb') as fp:
        for i in range(info['docfreq']):
            fp.seek(info['pos']+(8*i))
            docinfo = unpack('if', fp.read(8))
            innerproduct = qw * docinfo[1]  # 선형결합(Linear) - a*b의 형태 => 분자에 들어감
            
            if docinfo[0] not in candidates:
                candidates[docinfo[0]] = innerproduct
            else:
                candidates[docinfo[0]] += innerproduct
            # 문서별로 vector의 한 element끼리의 곱의 합 => Linear Sum
            # element-wise, dot, linear sum 다 같은 말인 것.
            # np.doc(q,d) => sum(a*b zip([el in q][el in d]))
            
for i, ip in candidates.items():
    # normalization => unit vector
    candidates[i] = ip/(sqrt(QLength)*sqrt(C[i]['length']))

# 코사인 유사도는 1이 제일 큰 값
res = sorted(candidates.items(), key=lambda c:c[1], reverse=True)[:K]
for i, r in enumerate(res):
    print('{}. {} ({})'.format((i+1), C[r[0]]['path'], r[1]))  # r[1] 이 가중치
    with open(C[r[0]]['path'], 'r', encoding='utf8') as fp:
        print('\n'.join(
            sent_tokenize(
                preprocessing1(fp.read(), p6, p5, p7, p5))[0:2]),
              end='\n\n')

1. news_naver/0003266491sid100.txt (0.15757162364287391)
북한이 16일 오전 윤석열 대통령의 한 일 정상회담 출국 2시간40분 전에 대륙간탄도미사일 을 발사했다.
윤 대통령은 출국에 앞서 긴급 국가안전보장회의 상임위원회에 참석해 북한의 무모한 도발은 분명히 대가를 치를 것 이라고 경고했다.

2. news_naver/0003266275sid100.txt (0.15134190108034148)
북한이 16일 쏜 대륙간탄도미사일 을 놓고 군 당국은 이날 예정된 한일 정상회담에 맞춰 화성-17형으로 계획된 도발에 나섰다고 분석했다.
지난달 화성-15형 발사 후 한 달이 채 안 된 시점에 북한이 카드를 다시 꺼내 들고 위협 수위를 높인 것이다.

3. news_naver/0013820555sid100.txt (0.14024450466222116)
전문가들 군사위성 준비 관측 미사일총국 지휘관 참관 운용주체 변화 시사 김정은 핵전쟁억제력 강화로 적들에 두려움 줘야 참관 평양 조선중앙통신 연합뉴스 김정은 북한 국무위원장이 전날 대륙간탄도미사일 화성-17형 발사훈련을 현지지도했다고 조선중앙통신이 17일 밝혔다.
중앙통신은 미국과 남조선 괴뢰역도들의 도발적이며 침략적인 대규모 전쟁연습소동으로 하여 조선반도 지역에 가장 불안정한 안전 환경이 조성되고 있는 엄중한 형세 하에서 조선로동당 중앙군사위원회는 3월 16일 대륙간탄도미싸일 화성포-17 형 발사 훈련을 단행하도록 하였다 고 보도했다.

4. news_naver/0001861821sid104.txt (0.12674725769954676)
북한이 대륙간탄도미사일을 개발할 수 있었던 배경에 강제노동과 주민착취 등 인권탄압이 있었다는 지적이 제기됐습니다.미국 고위당국자는 유엔 안전보장이사회의 북한인권 관련 비공식 회의를 앞두고 전화 브리핑에서 북한 정권이 대량살상무기 개발 자금을 조달할 수 있었던 것은 강제노동과 착취가 있었기 때문이라고 말했습니다.그러면서 북한 주민들은 극심한 고난과

In [ ]:
원래는 DTM = 
   t1 t2 t3 ... tn ㅌ V
d1 vector => 행벡터(굉장히 sparse 하고 큼. high dim.) -> 한 문서에 모든 단어가 나올 수 없기 때문에
d2
d3
..
dm
q  vector =>
Query를 동일한 벡터 공간상에 표현.
여기서 bottle neck 현상이 발생(비교를 다 해야하기 때문). 그래서 transpose!

TDM -> 이게 문서끼리 비교한 것. 어차피 만들어지는 것은 vector, matrix
   d1, d2, d3, ... dm q   => 이게 문서, 문장, 토픽, 카테고리, 프로파일, ... 여러가지가 될 수 있음. 그래서 다양하게 사용 가능
t1 w1                 *
t2 w2 
t3 w3                 *
{t ㅌ Q} vector = Projection
{t1, t3 ㅌ Q}, {t1, t3 ㅌ d_1..m ㅌ C}
..
tn
                      => 열벡터
    
Consine Sim. => dot => 어느 한쪽이 0이면 0이 나옴.
|q 벡터| < |d|  => 따라서 TDM 이 DTM 보다 row
전체 doc 벡터를 구할 필요가 없다!
q 안에 들어있는 애만 보겠다!

In [50]:
len(QTM)

927

In [ ]:
지금 우리가 한 것.
repr(query), repr(docㅌC)  벡터들 비교 => Consine=Similarity=Relevance
같은 VectorSpace 공간 상에 표현하고 비교한 것. 특히, 코사인 유사도를 썼기 때문에, Query 벡터에 Projection 시킨 것.
원래 전체 Dims. 이었을텐데, 코사인 유사도 구할 때 Query 벡터가 0인 것을 내적하면 어차피 값이 0이 됨. 
따라서 Query 벡터에 있는 term 만으로 차원을 줄인 것. 즉 Dims. => V => Q
t in Q ㅌ V 일 때, Dim = V가 아니라, Dim = tㅌQ.  즉 Q 벡터에 오버래핑한 것.
Query 벡터를 구성하는 차원으로만 각각의 문서들을 Projection 시킨 것.

=> repr(Term/Token)
TDM 의 행벡터 비교. 단어끼리 비교할 수 있을 것.

In [ ]:
repr(Term) = {d1, d2, d3, ... ,dn ㅌ C}
repr(Q(newtoken)) = Dims(Doc.)

In [51]:
TDM['0']

{'docfreq': 9, 'pos': 0}

In [53]:
V[7]

'도쿄'

In [54]:
info = TDM['7']
with open('weights.dat', 'rb') as fp:
    for i in range(info['docfreq']):
        fp.seek(info['pos']+(8*i))
        docw = unpack('if', fp.read(8))   # 여기까지 TDM['7']에 대한 벡터표현

In [55]:
info = TDM['7']
termVector = dict()
with open('weights.dat', 'rb') as fp:
    for i in range(info['docfreq']):
        fp.seek(info['pos']+(8*i))
        docw = unpack('if', fp.read(8))
        termVector[docw[0]] = docw[1]

In [56]:
termVector

{409: 0.331951767206192,
 406: 0.2547221779823303,
 401: 0.06893254816532135,
 397: 0.2174457460641861,
 351: 0.292304128408432,
 332: 0.14226503670215607,
 330: 0.2174457460641861,
 322: 0.45719364285469055,
 319: 0.35458484292030334,
 312: 0.15920135378837585,
 308: 0.16308431327342987,
 301: 0.36388880014419556,
 293: 0.2174457460641861,
 272: 0.3962344825267792,
 260: 0.20895177125930786,
 252: 0.38053005933761597,
 240: 0.3876206874847412,
 234: 0.8154215812683105,
 233: 0.5094443559646606,
 232: 0.20261991024017334,
 186: 0.8303443193435669,
 183: 0.7610601186752319,
 179: 0.3876206874847412,
 178: 0.20261991024017334,
 166: 0.12736108899116516,
 159: 1.1608432531356812,
 147: 0.24002665281295776,
 144: 0.21226847171783447,
 140: 0.18194440007209778,
 127: 0.543614387512207,
 126: 0.4052398204803467,
 122: 0.14379477500915527,
 87: 0.09051041305065155,
 82: 0.20261991024017334,
 81: 0.2328616827726364,
 77: 0.3428952097892761,
 64: 0.4348914921283722,
 63: 0.4348914921283722,
 34

In [57]:
V[12]

'일본'

In [58]:
# t = {d1, d2, ..., dm=0 X}
# 어차피 사용안된 문서는 곱해도 0이라서 표현 안해도 상관 전혀 없음
# 다른 term 들에 대한 vector 구해서 벡터끼리 비교
# 이 벡터를 TDM{docfreq}+Posting/Weights
# 즉 다른 t들에 대한 v <= TDM{docfreq}+Posting/Weighting

info = TDM['12']  # docfreq 가 작아서 작은 차원.
termVector = dict()
with open('weights.dat', 'rb') as fp:
    for i in range(info['docfreq']):
        # (docIdx, weight)
        fp.seek(info['pos']+(8*i))
        docw = unpack('if', fp.read(8))
        termVector[docw[0]] = docw[1]

result = dict()
for j, info in TDM.items():
    with open('weights.dat', 'rb') as fp:
        for i in range(info['docfreq']):
            fp.seek(info['pos']+(8*i))
            docw = unpack('if', fp.read(8))
            
#            t ㅌ Q AND t ㅌ Candi  => 어차피 쿼리에 없으면, 곱해봤자 0일 거니까 곱할 필요도 없으니 걸러내자.
#            element 를 weight 로 삼은 것.
            if docw[0] in termVector:
                ip = termVector[docw[0]] * docw[1]
                if j not in result:
                    result[j] = {'ip':ip, 'len':docw[1]**2}
                else:
                    result[j]['ip'] += ip   # linear sum
                    result[j]['len'] += docw[1]**2
                    

candidate = dict()
for j, info in result.items():
    if sqrt(info['len'])>0:
        candidate[V[int(j)]] = info['ip'] / sqrt(info['len'])

In [59]:
result

{'0': {'ip': 0.5849149793049412, 'len': 0.059622008068656085},
 '1': {'ip': 0.844548425061559, 'len': 0.09348146667009018},
 '2': {'ip': 1.520509448382462, 'len': 0.24386833423308463},
 '3': {'ip': 1.2336381740184903, 'len': 0.2111063998442016},
 '4': {'ip': 10.476402452830868, 'len': 6.506633819403311},
 '5': {'ip': 2.2662007279786707, 'len': 0.47135395325374135},
 '6': {'ip': 0.5191066733255297, 'len': 0.05573871753931017},
 '7': {'ip': 9.910022770000468, 'len': 7.0213631953829525},
 '8': {'ip': 0.37891208956145395, 'len': 0.06430090128627525},
 '9': {'ip': 2.3353094536489185, 'len': 0.4360290174787467},
 '10': {'ip': 2.3050171409572906, 'len': 1.4191934635637073},
 '11': {'ip': 0.4260870152353884, 'len': 0.08642351798010595},
 '12': {'ip': 31.32939723715883, 'len': 31.32939723715883},
 '13': {'ip': 2.25208239037187, 'len': 2.6202973479324942},
 '14': {'ip': 0.227815056755722, 'len': 0.03222767457529635},
 '15': {'ip': 0.18875178091458888, 'len': 0.027951216552355718},
 '16': {'ip': 

In [60]:
V[27], sqrt(sum([_**2 for _ in list(termVector.values())]))
# 나눠서 1되는지 확인

('방일', 5.597266943532247)

In [61]:
sorted(candidate.items(), key=lambda r:r[1], reverse=True)[:20]

[('일본', 5.597266943532248),
 ('일본/NNP', 5.582609957414418),
 (' 일', 5.2802939475008905),
 ('본 ', 5.236562232656297),
 (' 한', 4.811788087967038),
 (' 대', 4.598384203826868),
 ('대통령', 4.5868116961061975),
 (' 총', 4.580799570671956),
 ('대통', 4.573086282230388),
 ('통령', 4.573086282230388),
 (' 윤', 4.568765303232235),
 ('대통령/NNG', 4.556554901598393),
 ('열 ', 4.502282530973823),
 ('총리/NNG', 4.3650981202967145),
 ('총리', 4.362068335833017),
 ('한국/NNP', 4.359515982973466),
 (' 관', 4.358474550679296),
 ('일 ', 4.345862470156733),
 ('한국', 4.310274040582356),
 ('하 았', 4.30236816118128)]

In [62]:
V[29]

'정치권'

In [63]:
info = TDM['29']
termVector = dict()
with open('weights.dat', 'rb') as fp:
    for i in range(info['docfreq']):
        fp.seek(info['pos']+(8*i))
        docw = unpack('if', fp.read(8))
        termVector[docw[0]] = docw[1]

result = dict()
for j, info in TDM.items():
    with open('weights.dat', 'rb') as fp:
        for i in range(info['docfreq']):
            fp.seek(info['pos']+(8*i))
            docw = unpack('if', fp.read(8))
            
#            t ㅌ Q AND t ㅌ Candi
            if docw[0] in termVector:
                ip = termVector[docw[0]] * docw[1]
                if j not in result:
                    result[j] = {'ip':ip, 'len':docw[1]**2}
                else:
                    result[j]['ip'] += ip
                    result[j]['len'] += docw[1]**2
                    

candidate = dict()
for j, info in result.items():
    if sqrt(info['len'])>0:
        candidate[V[int(j)]] = info['ip'] / sqrt(info['len'])

In [64]:
sorted(candidate.items(), key=lambda r:r[1], reverse=True)[:20]

[('정치권', 0.2554615099053896),
 ('월/NNB', 0.251493531136509),
 (' 쏠', 0.251493531136509),
 ('두에', 0.251493531136509),
 ('하 며', 0.2481992631978625),
 ('하며', 0.2481992631978625),
 ('네이버', 0.24817760842006878),
 ('는 언론사', 0.24797212429110463),
 ('편집', 0.2478663566549363),
 ('편집/NNP', 0.24747252299306466),
 ('언론사 편집', 0.24747252299306466),
 (' 네', 0.24703592533594976),
 ('네이버/NNP', 0.24678222631543445),
 ('네이', 0.24678222631543445),
 ('이버', 0.24678222631543445),
 ('구독중', 0.24660630897047803),
 ('구독 중', 0.24660630897047803),
 ('독중', 0.24660630897047803),
 ('페이지', 0.2463185418787577),
 ('언론사/NNG', 0.24609632176185497)]

In [ ]:
# 우리가 independent 하다는 가정을 세워도, 연관성있는 애들이 튀어나오기는 하구나 라는 것을 느끼면 됨.
# doc 차원을 늘리면, 유니크한 term 개수가 늘어나니까 term 차원의 수도 늘어남.
# 앞에서 preprocessing 과정에서 잘 하면 더 정제되어 나올 것.

In [ ]:
'https://nlp.stanford.edu/IR-book/'

In [ ]:
C = {d1ㅌc1, d2ㅌc2, ..., dnㅌck}
kNN = {d1,d2,...,d8}
    = {c1,c2,...,c_j}
k=짝수 안됨 => 동수 나와서. 그래서 일반적으로 홀수 씀

k=5 (c1=3,c2=1,c3=1)
len(NN) == 5
(c1 AND d ㅌ C)/k
return argmaxj prob(3/5)
class = 1

In [ ]:
문서마다 카테고리값만 잘 붙어 있으면 그 상태로 바로 kNN 할 수 있음

In [65]:
# inner product
K = 10
candidates = dict()

for t, qw in QTM.items():  # 쿼리에 대한 가중치 들어있음
    j = str(V.index(t))
    info = TDM[j]
    
    # 각 단어에 있는 문서의 웨이트들 돌면서,
    with open('weights.dat', 'rb') as fp:
        for i in range(info['docfreq']):
            fp.seek(info['pos']+(8*i))
            docinfo = unpack('if', fp.read(8))
            innerproduct = qw * docinfo[1]  # 선형결합(Linear) - a*b의 형태 => 분자에 들어감
            
            if docinfo[0] not in candidates:
                candidates[docinfo[0]] = innerproduct
            else:
                candidates[docinfo[0]] += innerproduct
            # 문서별로 vector의 한 element끼리의 곱의 합 => Linear Sum
            # element-wise, dot, linear sum 다 같은 말인 것.
            # np.doc(q,d) => sum(a*b zip([el in q][el in d]))
            
for i, ip in candidates.items():
    # normalization => unit vector
    candidates[i] = ip/(sqrt(QLength)*sqrt(C[i]['length']))

# 코사인 유사도는 1이 제일 큰 값
kNN = dict()
res = sorted(candidates.items(), key=lambda c:c[1], reverse=True)[:K]
for i, r in enumerate(res):
    k = re.search('(\d{3})[.]txt$', C[r[0]]['path']).group(1)
    if k not in kNN:
        kNN[k] = 0
    kNN[k] += 1
    print('{}. {} ({})'.format((i+1), C[r[0]]['path'], r[1]))  # r[1] 이 가중치
    with open(C[r[0]]['path'], 'r', encoding='utf8') as fp:
        print('\n'.join(
            sent_tokenize(
                preprocessing1(fp.read(), p6, p5, p7, p5))[0:2]),
              end='\n\n')

1. news_naver/0003266491sid100.txt (0.15757162364287391)
북한이 16일 오전 윤석열 대통령의 한 일 정상회담 출국 2시간40분 전에 대륙간탄도미사일 을 발사했다.
윤 대통령은 출국에 앞서 긴급 국가안전보장회의 상임위원회에 참석해 북한의 무모한 도발은 분명히 대가를 치를 것 이라고 경고했다.

2. news_naver/0003266275sid100.txt (0.15134190108034148)
북한이 16일 쏜 대륙간탄도미사일 을 놓고 군 당국은 이날 예정된 한일 정상회담에 맞춰 화성-17형으로 계획된 도발에 나섰다고 분석했다.
지난달 화성-15형 발사 후 한 달이 채 안 된 시점에 북한이 카드를 다시 꺼내 들고 위협 수위를 높인 것이다.

3. news_naver/0013820555sid100.txt (0.14024450466222116)
전문가들 군사위성 준비 관측 미사일총국 지휘관 참관 운용주체 변화 시사 김정은 핵전쟁억제력 강화로 적들에 두려움 줘야 참관 평양 조선중앙통신 연합뉴스 김정은 북한 국무위원장이 전날 대륙간탄도미사일 화성-17형 발사훈련을 현지지도했다고 조선중앙통신이 17일 밝혔다.
중앙통신은 미국과 남조선 괴뢰역도들의 도발적이며 침략적인 대규모 전쟁연습소동으로 하여 조선반도 지역에 가장 불안정한 안전 환경이 조성되고 있는 엄중한 형세 하에서 조선로동당 중앙군사위원회는 3월 16일 대륙간탄도미싸일 화성포-17 형 발사 훈련을 단행하도록 하였다 고 보도했다.

4. news_naver/0001861821sid104.txt (0.12674725769954676)
북한이 대륙간탄도미사일을 개발할 수 있었던 배경에 강제노동과 주민착취 등 인권탄압이 있었다는 지적이 제기됐습니다.미국 고위당국자는 유엔 안전보장이사회의 북한인권 관련 비공식 회의를 앞두고 전화 브리핑에서 북한 정권이 대량살상무기 개발 자금을 조달할 수 있었던 것은 강제노동과 착취가 있었기 때문이라고 말했습니다.그러면서 북한 주민들은 극심한 고난과

In [66]:
{k:v/sum(kNN.values()) for k, v in kNN.items()}
# 이게 posting data 에 학습되어 있으니, 앞으로 어떤 query가 들어가도 분류할 것.

{'100': 0.6, '104': 0.3, '809': 0.1}

In [68]:
Q = '''
홍 시장은 이날 오전 CBS라디오 ‘김현정의 뉴스쇼’ 인터뷰에서 “원래 당 지도부라는 건 그렇다. 총선을 치르려면 물갈이 공천을 해야 되는데, 물갈이 공천을 하려면 (지도부) 본인이 불출마 선언을 해야 된다”고 말했다.

홍 시장은 “제가 2011년도 당대표를 할 때 저하고 관련 없고, 우리하고 관련 없는 디도스 파동으로 결국은 스스로 물러났다”며 “2012년도 1월이 되면 내 스스로가 이번 선거에 불출마한다고 하고 문제되는 사람들 전부 끌고 나가려고 했다”고 당시 상황을 설명했다. 이어 “그게 외부로 알려지는 바람에 온갖 중진들이 다 들고 일어섰다”며 “벌떼처럼 대들어서 더 이상 당대표 하는 게 무의미하다는 생각으로 그 당시에 사퇴를 했는데, 그게 겉으로 포장은 디도스 파동이었다”고 말했다.

홍 시장은 “물갈이 공천을 하려면 제일 먼저 해야 되는 게 지도부에 있는 사람들이 불출마 선언을 하고 지금부터 물갈이 공천하자, 그런 식으로 정리해 나가는 게 혁신적인 방법”이라며 “그 방법을 지금 할 수 있겠냐”고 반문했다. 그는 “그런 비상상황이라도 만들고 대처하지 않으면 내년 총선이 상당히 어렵지 않겠느냐”며 “그렇게라도 충격을 안 주면 소위 지도부라는 사람들이 각성하겠냐”고 말했다.

또 그는 “지금 상황으로 가면 여소야대가 바뀌기가 어렵다”며 “당이 한 마음이 돼서 총선에 임해야 되는데 지금 당이 돌아가는 모습을 보면 한마음이 되기가 상당히 힘든 구조로 가고 있다”고 우려했다. 그러면서 “당내 이간질하는 세력들하고도 어떤 스탠스로 당을 만들어갈지 정리가 안 돼 있다”며 “이간질을 해서 자기 존재를 부각시키려고 하는 사람들”이라고 지적했다. 이어 “지도부가 용산의 눈치나 보고 하명만 기다리고 이런 식으로 당 운영이 돼서는 안 된다”며 “지도부가 카리스마가 있어야 된다”고 당부했다.

한편 홍 시장은 미국 정보기관이 한국 등 동맹국을 도·감청한 정황이 공개된 것을 놓고 여야 불문 미국의 사과를 촉구하는 상황과 관련해 이날 “외교 비사에 속하는 거지 외교 정사의 문제는 아니다”라고 말했다.
'''

# DL, ML = dataloader(yield, preprocessing)
d = preprocessing1(Q, p1, p2, p3, p4, p6, p5, p7, p5)
tokens = preprocessing2(d)
# 내가 학습을 이렇게 시켰으니 앞으로 올 애들도 ㄱ똑같은 처리를 해줘야 같은 벡터 공간에 생길 것.

QTM = Counter(filter(lambda t:t in V, preprocessing3(tokens)))
maxfreq = max(QTM.values())
QLength = 0

for t, freq in QTM.items():
    TF = tf6(0, freq, maxfreq)  # k를 0.5로 바꿀 수 있음. 이전 결과가 1이 나옴을 확인했기 때문. 0.5를 주면 빈도의 영향을 덜 받게 됨
    # 그래서 IDF 의 영향력을 키워주는 것.
    IDF = idf2(TDM[str(V.index(t))]['docfreq'], len(C))
    W = TF*IDF
    QLength += W**2
    QTM[t] = W

In [69]:
K = 10
candidates = dict()

for t, qw in QTM.items():  # 쿼리에 대한 가중치 들어있음
    j = str(V.index(t))
    info = TDM[j]
    
    # 각 단어에 있는 문서의 웨이트들 돌면서,
    with open('weights.dat', 'rb') as fp:
        for i in range(info['docfreq']):
            fp.seek(info['pos']+(8*i))
            docinfo = unpack('if', fp.read(8))
            innerproduct = qw * docinfo[1]  # 선형결합(Linear) - a*b의 형태 => 분자에 들어감
            
            if docinfo[0] not in candidates:
                candidates[docinfo[0]] = innerproduct
            else:
                candidates[docinfo[0]] += innerproduct
            # 문서별로 vector의 한 element끼리의 곱의 합 => Linear Sum
            # element-wise, dot, linear sum 다 같은 말인 것.
            # np.doc(q,d) => sum(a*b zip([el in q][el in d]))
            
for i, ip in candidates.items():
    # normalization => unit vector
    candidates[i] = ip/(sqrt(QLength)*sqrt(C[i]['length']))

# 코사인 유사도는 1이 제일 큰 값
kNN = dict()
res = sorted(candidates.items(), key=lambda c:c[1], reverse=True)[:K]
for i, r in enumerate(res):
    k = re.search('(\d{3})[.]txt$', C[r[0]]['path']).group(1)
    if k not in kNN:
        kNN[k] = 0
    kNN[k] += 1
    print('{}. {} ({})'.format((i+1), C[r[0]]['path'], r[1]))  # r[1] 이 가중치
    with open(C[r[0]]['path'], 'r', encoding='utf8') as fp:
        print('\n'.join(
            sent_tokenize(
                preprocessing1(fp.read(), p6, p5, p7, p5))[0:2]),
              end='\n\n')

1. news_naver/0000032362cid1089510.txt (0.185618080171489)
나를 위한 정치 해설 정치왜그래 는 매주 화요일 저녁 7시 시사 유튜브에서 방송됩니다.
뉴스를 보는 또 다른 관점과 정보를 제공합니다.

2. news_naver/0004162607cid1089303.txt (0.14396830239010153)
책임정치를 위한 선거제도 개혁 방향은 김재훈 선임연구위원 서울경제 국회 의석을 169석 장악한 거대 야당이 범죄에 연루될 가능성이 있는 당 대표의 구속을 막기 위해 구속 여부에 대한 법원의 판단도 받지 않겠다며 체포동의안을 부결시켰다.
거야는 경제 사회 안보 외교의 복합 위기로 고통받고 있는 국민과 기업이 경제 회생을 위한 정책 입법을 고대하고 있음에도 대통령실이나 여당의 입법 협조 요청은 외면한 채 특검 등으로 대통령 공격에만 집중하고 있다.

3. news_naver/0001862105sid100.txt (0.1252478286536311)
진행 이광연 앵커 박석원 앵커 출연 양기호 성공회대 일본학과 교수 아래 텍스트는 실제 방송 내용과 차이가 있을 수 있으니 보다 정확한 내용은 방송으로 확인하시기 바랍니다.
인용 시 뉴스 명시해주시기 바랍니다.

4. news_naver/0001862159sid100.txt (0.11426587741719699)
진행 함형건 앵커 출연 김재섭 국민의힘 도봉구 갑 당협위원장 서용주 더불어민주당 상근부대변인 아래 텍스트는 실제 방송 내용과 차이가 있을 수 있으니 보다 정확한 내용은 방송으로 확인하시기 바랍니다.
인용 시 24 명시해주시기 바랍니다.

5. news_naver/0000032356cid1089510.txt (0.11033667673607335)
정치왜그래 나를 위한 정치 해설 정치왜그래 는 매주 화요일 저녁 7시 시사 유튜브에서 방송됩니다.
뉴스를 보는 또 다른 관점과 정보를 제공합니다.

6. news_naver/0000588404cid1089448.txt (0.1

In [70]:
{k:v/sum(kNN.values()) for k, v in kNN.items()}
# 이게 posting data 에 학습되어 있으니, 앞으로 어떤 query가 들어가도 분류할 것.

{'510': 0.3, '303': 0.1, '100': 0.3, '448': 0.3}

In [ ]:
# 옛날의 거를 지금와서 분류해도 훌륭한 수준임.

In [ ]:
# 이렇게해서 지난주부터 오늘까지 텍스트마이닝의 가장 큰 축인 정보검색
# 벡터표현, 벡터공간 구성, 벡터공간 축(BoW - independent, 극복을 위한 n-gram preprocessing)
# 임베딩해서 단어도, 문서도 유사한 애들 찾아보고
# 한발 더 나가서 KNN 이라는 분류 알고리즘까지 돌려본 것.